In [1]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
import pandas as pd
import numpy as np

2024-02-06 19:57:33.159176: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 19:57:33.184828: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 19:57:33.184852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 19:57:33.184865: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-06 19:57:33.189958: I tensorflow/core/platform/cpu_feature_g

In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


2024-02-06 19:57:34.680934: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:57:34.684404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:57:34.684428: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:57:34.687452: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-06 19:57:34.687498: I tensorflow/compile

In [3]:
df = pd.read_csv(['../../../datos/july_23/2023-07-01.csv', "../../../datos/july_23/2023-07-02.csv", '../../../datos/july_23/2023-07-03.csv', "../../../datos/july_23/2023-07-04.csv"])

In [4]:
(df["from_address"] == df["to_address"]).sum()

2108

In [5]:
df = df[df["from_address"] != df["to_address"]]

In [6]:
(df["from_address"] == df["to_address"]).sum()

0

In [7]:
df.columns

Index(['block_timestamp', 'nonce', 'from_address', 'to_address', 'value',
       'gas'],
      dtype='object')

In [8]:
len(set(df['from_address']).union(set(df['to_address'])))

310475

In [9]:
df = df[~df.to_address.isna()]
df = df[~df.from_address.isna()]

In [10]:
ids = {}

for i, id in enumerate(set(df['from_address']).union(set(df['to_address']))):
    ids[id] = i

In [11]:
ids

{'0x249c914b4dcf9870439d28c0aa6f14b0c9cfc658': 0,
 '0xab66a9225ea6051a8f358c054ac8dfc9f39eae8a': 1,
 '0x1c61c2b975a227206b9a21f7d4486ec2312cd184': 2,
 '0xe996f8410a85861bde931350b066637bf616015c': 3,
 '0xfea87bed7c6df111ba46f55df42ec7c1522248f8': 4,
 '0x7c780434a6c298bd231f0ceab9dceb7bea627c17': 5,
 '0x0436d6a20bca2cfca1c1f996763f4ca6fca8895e': 6,
 '0xf62f5892aa0aec0d10bd4c2ef4630c1d6c1ec007': 7,
 '0x34bcafa4fe7fef6375db1d50f34357523c9a3521': 8,
 '0xe97342843c222b6865aecd4ced95bed2e48c0fc4': 9,
 '0x6b3b73bcfcf94418a8893e58ea3cdd04f381d5f0': 10,
 '0x5baf9bb3a195a354aefa8ad29653440e84116584': 11,
 '0x0d66bdeda138a236d1c2ff3382780f4e9ee50437': 12,
 '0xc99e74246984843d4eb5b55be0deab2139163bf5': 13,
 '0xc2901a6d74ac85cde60a9d8cae83695d1262b1df': 14,
 '0x5555d175816adc10f5a9f16a5db161ba5b24ca9b': 15,
 '0xa780f1ea8437599041c19d11818a6da0a6cf25bf': 16,
 '0xbc85298d3d52c3517702e5cec385d9c54b8d8998': 17,
 '0x4536f3731818eee0dca93919e4bc30c1e34de98b': 18,
 '0xcbe7d0871a9fa8cddf9f3bdbf3451c2fa5414

In [12]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, BatchNormalization, Lambda

# el 1 representa a cualquier indice, un tensor de una dimensión
input_aux = Input(1)

# el 128 es arbitrario, podría ser de cualquier valor
x = Embedding(len(ids), 128)(input_aux)

# TODO: investigar que es esto
x = Dense(64, activation='tanh')(x)

# lleva todos los valores entre -1 y 1
output_aux = BatchNormalization()(x)

model_aux = Model(input_aux, output_aux)
model_aux.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 embedding (Embedding)       (None, 1, 128)            39740288  
                                                                 
 dense (Dense)               (None, 1, 64)             8256      
                                                                 
 batch_normalization (Batch  (None, 1, 64)             256       
 Normalization)                                                  
                                                                 
Total params: 39748800 (151.63 MB)
Trainable params: 39748672 (151.63 MB)
Non-trainable params: 128 (512.00 Byte)
_________________________________________________________________


In [13]:
# son los 3 inputs que representan las 3 cabezas de la red, lo que se conoce como siamesa
input_layer_anchor = Input(1)
input_layer_positive = Input(1)
input_layer_negative = Input(1)

x_a = model_aux(input_layer_anchor)
x_p = model_aux(input_layer_positive)
x_n = model_aux(input_layer_negative)

# un tensor es ... investigar bien. Puede ser cualquier cosa, literalmente.
merged_output = Lambda(lambda tensors: tf.stack(tensors, axis=-1))([x_a, x_p, x_n])

model = Model([input_layer_anchor, input_layer_positive, input_layer_negative], merged_output)

In [14]:
from keras.utils import Sequence
import numpy as np

class GeneratorTriplet(Sequence):
    def __init__(self, df, ids, batch_size):
        self.df = df
        self.act_index = 0
        self.ids = ids
        self.batch_size = batch_size
        self.limit = int(np.ceil(len(self.df) / self.batch_size))

    def __len__(self):
        return int(np.ceil(len(self.df) / self.batch_size))
    
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.act_index < self.limit:
            resultado = self.__getitem__(self.act_index)
            self.act_index += 1
            return resultado
        else:
            raise StopIteration

    def __getitem__(self, index):
        # el Anchor es un A, el positive es un B y el negative es un C. Queremos acercar A y B tanto como sea posible alejando a C. Podría suceder que sea un vecino, pero la probabilidad es baja porque es al azar.
        # Si llegara a elegir algún vecino mal, de casualidad, en una siguiente epoch debería corregirse o inclusive en una misma epoch en una siguiente iteración del generador.
        init = index * self.batch_size
        end = (index + 1) * self.batch_size

        # el batch van a ser #batch_size tuplas que representan el anchor y positive. El negative es un random.
        batch = self.df[init:end]

        # me agarro un sample de #batch_size transacciones.
        negative = self.df.sample(len(batch))

        anchor = np.array(batch['from_address'].apply(lambda x: self.ids.get(x)))
        positive = np.array(batch['to_address'].apply(lambda x: self.ids.get(x)))
        negative = np.array(negative['to_address'].apply(lambda x: self.ids.get(x)))
        
        anchor = tf.convert_to_tensor(anchor)
        positive = tf.convert_to_tensor(positive)
        negative = tf.convert_to_tensor(negative)
        
        # el fake target simunla lo que sería aprendizaje supervisado        
        fake_target = tf.convert_to_tensor(np.array([1]*self.batch_size))

        return ([anchor, positive, negative], [fake_target]*3)

In [15]:
generator = GeneratorTriplet(df, ids, 64)

In [16]:
class TripletCustom(tf.keras.losses.Loss):
    def __init__(self, margin=0.2, **kwargs):
        super().__init__(**kwargs)
        self.margin = margin

    def __call__(self, y, y_pred, sample_weight=None):
        anchor, positive, negative =  tf.split(y_pred, num_or_size_splits=3, axis=-1)
        distance_positive = tf.reduce_sum(tf.square(anchor - positive), axis=1)
        distance_negative = tf.reduce_sum(tf.square(anchor - negative), axis=1)

        loss = tf.maximum(distance_positive - distance_negative + self.margin, 0.0)
        return tf.reduce_mean(loss)

loss = TripletCustom()    

model.compile(
    optimizer=Adam(2e-4),
    loss=loss
)

In [17]:
model.fit(generator, epochs=10)

Epoch 1/10


2024-02-06 19:57:40.010971: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fa5f2f69500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-06 19:57:40.011020: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2024-02-06 19:57:40.014742: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-06 19:57:40.150903: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-06 19:57:40.210806: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


8009/8009 [==============================] - 130s 16ms/step - loss: 0.3777
Epoch 2/10
8009/8009 [==============================] - 124s 16ms/step - loss: 0.1973
Epoch 3/10
8009/8009 [==============================] - 121s 15ms/step - loss: 0.0988
Epoch 4/10
8009/8009 [==============================] - 123s 15ms/step - loss: 0.0402
Epoch 5/10
8009/8009 [==============================] - 122s 15ms/step - loss: 0.0111
Epoch 6/10
8009/8009 [==============================] - 122s 15ms/step - loss: 0.0017
Epoch 7/10
8009/8009 [==============================] - 122s 15ms/step - loss: 5.5126e-04
Epoch 8/10
8009/8009 [==============================] - 125s 16ms/step - loss: 4.6757e-04
Epoch 9/10
8009/8009 [==============================] - 122s 15ms/step - loss: 4.3451e-04
Epoch 10/10
8009/8009 [==============================] - 123s 15ms/step - loss: 4.1164e-04


In [18]:
# input_layer = Input(shape=(2,))
# dense_layer = Dense(units=1, activation='linear')(input_layer)
# model = Model(inputs=input_layer, outputs=dense_layer)
# model.compile(optimizer='adam', loss='mean_squared_error')
# model.summary()
# y_train = np.random.rand(100)
# print(model.fit(combined_tensor, y_train, epochs=1, batch_size=32))